<br><br><br><br><br><br><br><br><br><br>

# Customer Churn Forecasting in Telecommunication Industry

<br>

<h1>1. Introduction</h1>

Considering the difference between costs of obtaining new customers and retaining existing customers, customer churn is a significant problem in business life.  
<br>
Companies struggle to launch special marketing campaigns in order to retain customers.  
<br>
Identifying customers which will leave the company soon may be a competitive advantage for companies.  
<br>
The aim of this analysis is to predict customer churn and identify customers which will leave the company soon since company wants to launch new marketing campaign for them in this scenario.  
<br>
Each row represents a customer, each column contains customers' attributes.  
The raw data contains 7043 rows (customers) and 21 columns (features).  
The 'Churn' column is our target.  
<br>
The data set includes information about:  

Customers who left within the last month – the column is called Churn  
Services that each customer has signed up for – phone, multiple lines, Internet, online security, online backup, device protection, tech support, and streaming TV and movies  
Customer account information – how long they have been a customer, contract, payment method, paperless billing, monthly charges, and total charges  
Demographic info about customers – gender, age range, and if they have partners and dependents  
<br>
* customerID : The Customer ID
* Gender : Whether the customer is a male or a female
* SeniorCitizen : Whether the customer is a senior citizen or not (1, 0)
* Partner : Whether the customer has a partner or not (Yes, No)
* Dependents : Whether the customer has dependents or not (Yes, No)
* tenure : Number of months the customer has stayed with the company
* PhoneService : Whether the customer has a phone service or not (Yes, No)
* MultipleLines : Whether the customer has multiple lines or not (Yes, No, No phone service)
* InternetService : Customer’s internet service provider (DSL, Fiber optic, No)
* OnlineSecurity : Whether the customer has online security or not (Yes, No, No internet service)
* OnlineBackup : Whether the customer has online backup or not (Yes, No, No internet service)
* DeviceProtection : Whether the customer has device protection or not (Yes, No, No internet service)
* TechSupport : Whether the customer has tech support or not (Yes, No, No internet service)
* StreamingTV : Whether the customer has streaming TV or not (Yes, No, No internet service)
* StreamingMovies : Whether the customer has streaming movies or not (Yes, No, No internet service)
* Contract : The contract term of the customer (Month-to-month, One year, Two year)
* Paperless : BillingWhether the customer has paperless billing or not (Yes, No)
* PaymentMethod : The customer’s payment method (Electronic check, Mailed check, Bank transfer, Credit card
* MonthlyCharges : The amount charged to the customer monthly
* TotalCharges : The total amount charged to the customer
* Churn : Whether the customer churned or not (Yes or No)

Importing libraries:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score, log_loss, precision_score, recall_score, f1_score, make_scorer

import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = None

Reading the data file:

In [2]:
raw_data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

The number of rows and columns of the data:

In [ ]:
raw_data.shape

The raw data contains 7043 rows (customers) and 21 columns (features).

Enabling Jupyter Notebook to show all columns in outputs and looking at the first/last 5 rows of data:

In [ ]:
raw_data.head()

In [ ]:
raw_data.tail()

<br>

***

<br>

<h1>2. Data Preparation</h1><br>

<h2>2.1. Identification of Variables & Data Types</h2>

In [ ]:
raw_data.info()

It can be understood that most of the variables have object data type, there are 2 variables which have int data type and there is 1 variable with float data type.

The number of distinct values of each column:

In [ ]:
raw_data.apply(lambda x: [x.nunique()])

The distinct values of each column:

In [ ]:
raw_data.apply(lambda x: [x.unique()])

Outputs illustrates that the most of the variables(18/21) are categorical and have 4 distinct values at most. 'Churn' is target variable and the other variables are predictors.

'tenure', 'MontlyCharges' and 'TotalCharges' are numerical variables of the data.

Although type of 'TotalCharges' should be float, it's type is object.  
<br>
It may have some string values.

In [ ]:
raw_data['TotalCharges'].sort_values().head(20)

There are 11 rows which contain empty values in 'TotalCharges' column.  
<br>
Thus, it's type will be transformed to float after missing value treatment process in Exploratory Data Analysis step.

<br><br>

<h2>2.2. Variable Transformations</h2>

First of all, 'customerID' column will be dropped since it is ineffective for the analysis.

In [3]:
data = raw_data.drop('customerID', axis=1)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
Gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), object(17)
memory usage: 1.1+ MB


Replacing 1 and 0 values in the 'MultipleLines' column with 'Yes' and 'No' since it is categorical feature.

In [5]:
data['SeniorCitizen'] = data['SeniorCitizen'].replace({1:'Yes',0:'No'})

Renaming 'tenure' column:

In [6]:
data = data.rename(columns={'tenure':'Tenure'})

In [7]:
data['MultipleLines'] = data['MultipleLines'].replace({'No phone service' : 'No'})

for i in range(8, 14):
    data.iloc[:, i] = data.iloc[:, i].replace({'No internet service' : 'No'})

<br>

***

<br>

<h1>3. Exploratory Data Analysis</h1><br>

<h2>3.1. Missing Value Treatment</h2>

Replacing empty values with na in the data:

Let's see the na values:

In [8]:
data = data.replace('', np.nan)
data = data.replace(' ', np.nan)

In [9]:
data[data.isnull().any(axis=1)]

,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,Female,No,Yes,Yes,0,No,No,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
753,Male,No,No,Yes,0,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,20.25,NaN,No
936,Female,No,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
1082,Male,No,Yes,Yes,0,Yes,Yes,No,No,No,No,No,No,No,Two year,No,Mailed check,25.75,NaN,No
1340,Female,No,Yes,Yes,0,No,No,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
3331,Male,No,Yes,Yes,0,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,19.85,NaN,No
3826,Male,No,Yes,Yes,0,Yes,Yes,No,No,No,No,No,No,No,Two year,No,Mailed check,25.35,NaN,No
4380,Female,No,Yes,Yes,0,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,20.00,NaN,No
5218,Male,No,Yes,Yes,0,Yes,No,No,No,No,No,No,No,No,One year,Yes,Mailed check,19.70,NaN,No
6670,Female,No,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,No


In [10]:
data['TotalCharges'] = data['TotalCharges'].fillna(0)

11 rows which have na value represent very small proportion in 7043 rows. Since tenure of the rows are 0, they can be filled with 0 to obtain missing value treatment.

In [11]:
data.isna().sum()

Gender              0
SeniorCitizen       0
Partner             0
Dependents          0
Tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Changing the data type of 'TotalCharges':

In [12]:
data['TotalCharges'] = data['TotalCharges'].astype(float)

<br><br>

<h2>3.2. Outlier Treatment</h2>

Identifying the outliers in continuous variables based on IQR Score Method:

In [ ]:
def find_outliers(data, column_list):
    for i in column_list:
        Q1 = data[i].quantile(0.25)
        Q3 = data[i].quantile(0.75)
        IQR = Q3 - Q1
        print(data[i][(data[i] < (Q1 - 1.5 * IQR)) | (data[i] > (Q3 + 1.5 * IQR))])
find_outliers(data, ['Tenure', 'MonthlyCharges', 'TotalCharges'])

It can be seen that columns do not contain any outliers.

<br><br>

<h2>3.3. Univariate Analysis</h2>


Firstly, target variable should be set and other variables should be split into 2 groups as categorical and numerical variables to make the analysis easier.

In [13]:
features = data.drop('Churn', axis=1)
target = pd.DataFrame(data['Churn'], columns=['Churn'])

In [14]:
nums = features[['Tenure', 'MonthlyCharges', 'TotalCharges']]
cats = features.drop(['Tenure', 'MonthlyCharges', 'TotalCharges'], axis=1)

<br><h3>3.3.1. Categorical</h3>

<b>Count%</b>

In [ ]:
round(target['Churn'].value_counts(normalize=True), 2)

The customers who stop doing business with the company represent %27 of the data.  
<br>
Hence, it can be said that this is an unbalanced data.

In [ ]:
for i in cats.columns:
    print(round(cats[i].value_counts(normalize=True), 2))
    print('\n')

<b>Pie Chart</b>

In [ ]:
for i in cats.columns:
    print('\n')
    print('\t'*2 + i)
    plt.pie(cats[i].value_counts(normalize = True), labels=cats[i].unique(), autopct='%1.1f%%', startangle=90)
    plt.axis('equal')
    plt.show()
    print('-'*45)

The outputs show that while some variables are balanced some are imbalanced. The imbalanced variables are as below:  
<br>
84% of the customers are not senior citizen.  
70% of the customers does not have dependents.  
71% of the customers does not have online security.  
71% of the customers does not have tech support.  
90% of the customers have phone service.

<br><h3>3.3.2. Numerical</h3>

In [ ]:
nums.describe()

In [ ]:
print('-----Variance-----')
for i in nums.columns:
    print(i + ': ' + str(round(nums[i].var(), 2)))

In [ ]:
print('-----Coefficient of Variation-----')
for i in nums.columns:
    print(i + ': ' + str(round((nums[i].std() / nums[i].mean()) * 100, 2)))

In [ ]:
nums.skew()

Since the outputs of the 'Tenure' and 'MonthlyCharges' variables are between -0.5 and 0 or 0 and 0.5 it can be said that these variables have symmetrical distribution. However, the output of the 'TotalCharges' variable is between 0.5 and 1, thus this variable is moderately (and positively) skewed.

In [ ]:
nums.kurtosis()

All variables have low kurtosis which is an indicator that data has light tails or lack of outliers. In addition, distribution is shorter, tails are thinner than the normal distribution.

<br>
<b>Histogram, Box Plot</b>

In [ ]:
nums.hist(bins=10, figsize=(15, 7))

In [ ]:
sns.boxplot(x=nums['Tenure'])

In [ ]:
sns.boxplot(x=nums['MonthlyCharges'])

In [ ]:
sns.boxplot(x=nums['TotalCharges'])

<br><br>

<h2>3.4. Bivariate Analysis</h2>

<h3>3.4.1. Categorical & Categorical</h3>

<b>Chi-Square Test</b>

In [ ]:
same = []
for i in cats.columns:
    cont = pd.crosstab(cats[i], target['Churn'])
    chi2_stat, p_val, dof, ex = stats.chi2_contingency(cont)
    print(i + ' - Churn')
    print("Chi2 Stat :" + str(chi2_stat))
    print("Critical Value :" + str(stats.chi2.ppf(q = 0.95, df = dof)))
    if chi2_stat < stats.chi2.ppf(q = 0.95, df = dof):
        same.append(i)
        print("The two distribution are the same.")
    else:
        print("Reject the null hypothesis that the two distribution are the same.")
    print("Degrees of Freedom :" + str(dof))
    print("P-Value :" + str(p_val))
    print("Contingency Table :" + str(ex))
    print('\n')

In [ ]:
same

According to the output, it can be understood that 'Gender' and 'PhoneService' are the only variables which have the same distribution since their chi-squared statistics exceeds the critical value.

<br><br>
<b>Bar Charts</b>

In [ ]:
def categorical_summarized(dataframe, x=None, y=None, hue=None, verbose=True):
    sns.countplot(x=x, y=y, hue=hue, data=dataframe)
    plt.show()

for i in cats.columns:
    categorical_summarized(data, x = i, hue='Churn')

<br><br>

<h3>3.4.2. Numerical & Numerical</h3>

<b>Correlation</b>

In [ ]:
sns.heatmap(nums.corr(), annot=True, cmap='YlGnBu')

<br><br>
<b>Scatter Plot</b>

In [ ]:
sns.set(style='ticks')

sns.pairplot(data)

<br><br>

<h3>3.4.3. Numerical & Categorical</h3>

In [ ]:
data['Churn'] = data['Churn'].replace({'Yes':1,'No':0})
data.groupby('Tenure')['Churn'].mean().sort_values(ascending=False).head(5)

In [ ]:
plt.plot(pd.DataFrame(data['Tenure'].unique(), columns=['Tenure']).sort_values(by='Tenure'), 
         data.groupby('Tenure')['Churn'].mean())
plt.show()

It can be seen that there is negative relationship between 'Tenure' and 'Churn'.

<br>
<b>Bar Charts</b>

In [ ]:
data['Churn'] = data['Churn'].replace({1:'Yes',0:'No'})

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(14, 4))
data[data['Churn'] == 'No'][['Tenure', 'MonthlyCharges', 'TotalCharges']].hist(bins=10, 
                                                                               color='green', 
                                                                               ax=ax)
data[data['Churn'] == 'Yes'][['Tenure', 'MonthlyCharges', 'TotalCharges']].hist(bins=10, 
                                                                                color='red', 
                                                                                ax=ax)

<br>

***

<br>

<h1>4. Data Preprocessing</h1><br>

<h2>4.1. Label Encoding</h2>

In [15]:
binary = []
multi = []
for i in cats.columns:
    if i in ['Gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']:
        binary.append(i)
    else:
        multi.append(i)

In [16]:
le = LabelEncoder()

target['Churn'] = le.fit_transform(target['Churn'])

for i in binary:
    features[i] = le.fit_transform(features[i])

<br><br>

<h2>4.2. Dummification</h2>

In [17]:
for i in multi:
    cols = []
    for j in list(features[i].unique()):
        cols.append(i + ':' + j)
    ohe = OneHotEncoder(dtype='int64', sparse=False)
    ohe_i = ohe.fit_transform(features[i].values.reshape(-1, 1))
    ohe_i = pd.DataFrame(ohe_i, columns=cols)
    features = pd.concat([features, ohe_i], axis=1)
    features = features.drop(i, axis=1)

<br><br>

<h2>4.3. Feature Scaling</h2>

Since there is no outlier in dataset, standartization is a good way to improve the performance of our model.

In [18]:
to_scale = ['TotalCharges', 'Tenure', 'MonthlyCharges']
std = StandardScaler()
scaled = std.fit_transform(features[to_scale])
scaled = pd.DataFrame(scaled, index=features.index, columns = to_scale)

In [19]:
features['TotalCharges'] = scaled['TotalCharges']
features['Tenure'] = scaled['Tenure']
features['MonthlyCharges'] = scaled['MonthlyCharges']

In [20]:
features.apply(lambda x: [x.unique()])

Gender                                                                          [[0.0, 1.0]]
SeniorCitizen                                                                   [[0.0, 1.0]]
Partner                                                                         [[1.0, 0.0]]
Dependents                                                                      [[0.0, 1.0]]
Tenure                                     [[-1.2774445836787656, 0.06632741908223598, -1...
PhoneService                                                                    [[0.0, 1.0]]
PaperlessBilling                                                                [[1.0, 0.0]]
MonthlyCharges                             [[-1.1603229160349193, -0.2596289419448806, -0...
TotalCharges                               [[-0.9926105235902257, -0.17216470898960817, -...
MultipleLines:No                                                                [[1.0, 0.0]]
MultipleLines:Yes                                                     

<br><br><br>

<h2>4.4. Train-Test Split</h2>

In [21]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=12)

***

<br>

<h1>5. Modeling</h1><br>

In [22]:
classifiers = {'Naive Bayes': GaussianNB(),
               'Decision Tree': DecisionTreeClassifier(criterion='entropy', random_state=13),
               'Logistic Reg.': LogisticRegression(random_state=12),
               'K-Nearset N.': KNeighborsClassifier(n_jobs=-1),
               'Linear SVC' : LinearSVC(random_state=14, max_iter=10000),
               'Random Forest' : RandomForestClassifier(criterion='entropy', n_jobs=-1, random_state=15),
               'Gradient B.T.': GradientBoostingClassifier(random_state=16)}

In [23]:
model_names = list(classifiers.keys())
accuracy = []
roc_auc = []
log_loss_list = []
f1_score_list = []
precision = []
recall = []

In [24]:
def results(y_test, y_pred):
    print('Accuracy Score :' + str(accuracy_score(y_test, y_pred)) + '\n')
    accuracy.append(accuracy_score(y_test, y_pred))
    print('Precision Score :' + str(precision_score(y_test, y_pred)) + '\n')
    precision.append(precision_score(y_test, y_pred))
    print('Recall Score :' + str(recall_score(y_test, y_pred)) + '\n')
    recall.append(recall_score(y_test, y_pred))
    print('ROC AUC :' + str(roc_auc_score(y_test,y_pred)) + '\n')
    roc_auc.append(roc_auc_score(y_test,y_pred))
    print('Log-Loss :' + str(log_loss(y_test, y_pred)) + '\n')
    log_loss_list.append(log_loss(y_test, y_pred))
    print('F1-Score :' + str(f1_score(y_test, y_pred)) + '\n')
    f1_score_list.append(f1_score(y_test, y_pred))
    print(classification_report(y_test, y_pred) + '\n')
    print('-----Confusion Matrix-----')
    print(confusion_matrix(y_test, y_pred))
    print('\n')

In [25]:
for clf in classifiers:
    model = classifiers[clf]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(clf, '\n')
    results(y_test, y_pred)
    print('-' * 100 + '\n')

Naive Bayes 

Accuracy Score :0.76

Precision Score :0.5191441441441441

Recall Score :0.7787162162162162

ROC AUC :0.7661613394987602

Log-Loss :8.289453185575422

F1-Score :0.622972972972973

              precision    recall  f1-score   support

           0       0.91      0.75      0.82      1733
           1       0.52      0.78      0.62       592

   micro avg       0.76      0.76      0.76      2325
   macro avg       0.71      0.77      0.72      2325
weighted avg       0.81      0.76      0.77      2325


-----Confusion Matrix-----
[[1306  427]
 [ 131  461]]


----------------------------------------------------------------------------------------------------

Decision Tree 

Accuracy Score :0.7329032258064516

Precision Score :0.4762684124386252

Recall Score :0.49155405405405406

ROC AUC :0.6534515798256421

Log-Loss :9.225305811792342

F1-Score :0.483790523690773

              precision    recall  f1-score   support

           0       0.82      0.82      0.82      1733


<br>

***

<br>

<h1>6. Model Evaluation</h1><br>

<h2>6.1. Performance Metrics</h2>

In [26]:
def second(x):
    return x[1]

<b>Accuracy</b>

In [27]:
for i, j in sorted(zip(model_names, accuracy), key=second, reverse=True):
    print(i + '\t' + ': {0:.{1}f}'.format(j, 3))

Gradient B.T.	: 0.811
Logistic Reg.	: 0.807
Linear SVC	: 0.806
Random Forest	: 0.785
K-Nearset N.	: 0.769
Naive Bayes	: 0.760
Decision Tree	: 0.733


<br><b>ROC AUC</b>

In [28]:
for i, j in sorted(zip(model_names, roc_auc), key=second, reverse=True):
    print(i + '\t' + ': {0:.{1}f}'.format(j, 3))

Naive Bayes	: 0.766
Gradient B.T.	: 0.725
Logistic Reg.	: 0.723
Linear SVC	: 0.721
K-Nearset N.	: 0.691
Random Forest	: 0.679
Decision Tree	: 0.653


<br><b>Log-Loss</b>

In [29]:
for i, j in sorted(zip(model_names, log_loss_list), key=second, reverse=False):
    print(i + '\t' + ': {0:.{1}f}'.format(j, 3))

Gradient B.T.	: 6.536
Logistic Reg.	: 6.670
Linear SVC	: 6.700
Random Forest	: 7.428
K-Nearset N.	: 7.992
Naive Bayes	: 8.289
Decision Tree	: 9.225


<br><b>F1-Score</b>

In [30]:
for i, j in sorted(zip(model_names, f1_score_list), key=second, reverse=True):
    print(i + '\t' + ': {0:.{1}f}'.format(j, 3))

Naive Bayes	: 0.623
Gradient B.T.	: 0.597
Logistic Reg.	: 0.593
Linear SVC	: 0.590
K-Nearset N.	: 0.539
Random Forest	: 0.524
Decision Tree	: 0.484


<br><b>Sensitivity</b>

In [31]:
for i, j in sorted(zip(model_names, recall), key=second, reverse=True):
    print(i + '\t' + ': {0:.{1}f}'.format(j, 3))

Naive Bayes	: 0.779
Logistic Reg.	: 0.552
Gradient B.T.	: 0.551
Linear SVC	: 0.549
K-Nearset N.	: 0.532
Decision Tree	: 0.492
Random Forest	: 0.465


<br><b>Precision</b>

In [32]:
for i, j in sorted(zip(model_names, precision), key=second, reverse=True):
    print(i + '\t' + ': {0:.{1}f}'.format(j, 3))

Gradient B.T.	: 0.652
Logistic Reg.	: 0.640
Linear SVC	: 0.639
Random Forest	: 0.600
K-Nearset N.	: 0.547
Naive Bayes	: 0.519
Decision Tree	: 0.476


The company wants to detect customers who will leave soon and offer them special marketing campaigns.  
<br>
Thus, precision is the most suitable option to evaluate the performance of models since the data is imbalance and the aim is to identify customers will leave as accurately as possible.

<br><br>
<h2>6.2. k-Fold Cross-Validation</h2>
<br>

In [33]:
cv_scores = []

for clf in classifiers:
    scores = []
    model = classifiers[clf]
    cv = KFold(n_splits=10, random_state=32)
    
    for train_index, test_index in cv.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    cv_scores.append(sum(scores) / len(scores))

for i, j in sorted(zip(model_names, cv_scores), key=second, reverse=True):
    print(i + '\t' + ': {0:.{1}f}'.format(j, 3))

Gradient B.T.	: 0.665
Logistic Reg.	: 0.655
Linear SVC	: 0.655
Random Forest	: 0.634
K-Nearset N.	: 0.568
Naive Bayes	: 0.523
Decision Tree	: 0.508


It would seem that Gradient Boosting Tree is the most suitable model for this case according to the precision metric.  
<br>
Next step is increasing performance of the model by trying different parameters.

<br><br>
<h2>6.3. Hyperparameter Tuning</h2>
<br>

In [34]:
kfold = KFold(n_splits=10, random_state=54)

for train_index, test_index in kfold.split(features.values):
    x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                        target.iloc[train_index], target.iloc[test_index]
        
    clf = GradientBoostingClassifier(random_state=99)
    
    parameters = {'n_estimators':range(20, 81, 10)}

    grid = GridSearchCV(clf, parameters, scoring='precision', cv=10, n_jobs=-1, iid=False)

    grid = grid.fit(x_train, y_train)

    model = grid.best_estimator_
    
    y_pred = model.predict(x_test)
    
    print(grid.best_params_, '{0:.{1}f}'.format(precision_score(y_test, y_pred), 4))

{'n_estimators': 20} 0.6560
{'n_estimators': 20} 0.7593
{'n_estimators': 20} 0.6863
{'n_estimators': 20} 0.6612
{'n_estimators': 20} 0.6604
{'n_estimators': 20} 0.6250
{'n_estimators': 20} 0.7426
{'n_estimators': 20} 0.6881
{'n_estimators': 20} 0.6637
{'n_estimators': 20} 0.7477


It can be seen that even though there is only one parameter tuned and others were at their default value, results vary significantly from 0.625 to 0.759 depending on variety among folds.

Thus, hyperparameter tuning will be done with k-fold cross-validation in order to maintain consistency of results.

In [35]:
for i in range(20, 81, 10):
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.1, 
                                           n_estimators=i,
                                           min_samples_split=40,
                                           min_samples_leaf=40,
                                           max_depth=5,
                                           max_features="sqrt",
                                           subsample=0.8,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

20 0.6878
30 0.6722
40 0.6672
50 0.6642
60 0.6613
70 0.6557
80 0.6570


In [36]:
for i in range(20, 81, 10):
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.05, 
                                           n_estimators=i,
                                           min_samples_split=40,
                                           min_samples_leaf=40,
                                           max_depth=5,
                                           max_features="sqrt",
                                           subsample=0.8,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

20 0.7833
30 0.7210
40 0.6958
50 0.6850
60 0.6777
70 0.6719
80 0.6673


In [37]:
for i in range(20, 81, 10):
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.01, 
                                           n_estimators=i,
                                           min_samples_split=40,
                                           min_samples_leaf=40,
                                           max_depth=5,
                                           max_features="sqrt",
                                           subsample=0.8,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

20 0.0000
30 0.0000
40 0.0000
50 0.0000
60 0.6500
70 0.8707
80 0.8547


In [38]:
for i in range(61, 80):
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.01, 
                                           n_estimators=i,
                                           min_samples_split=40,
                                           min_samples_leaf=40,
                                           max_depth=5,
                                           max_features="sqrt",
                                           subsample=0.8,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

61 0.6556
62 0.6386
63 0.8417
64 0.8931
65 0.8876
66 0.8812
67 0.8603
68 0.8628
69 0.8686
70 0.8707
71 0.8802
72 0.8773
73 0.8689
74 0.8732
75 0.8732
76 0.8540
77 0.8602
78 0.8650
79 0.8684


In [39]:
for i in range(5, 12):
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.01, 
                                           n_estimators=64,
                                           min_samples_split=40,
                                           min_samples_leaf=40,
                                           max_depth=i,
                                           max_features="sqrt",
                                           subsample=0.8,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

5 0.8931
6 0.8667
7 0.8768
8 0.8676
9 0.8703
10 0.8768
11 0.8648


In [40]:
for i in range(40, 301, 20):
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.01, 
                                           n_estimators=64,
                                           min_samples_split=i,
                                           min_samples_leaf=40,
                                           max_depth=5,
                                           max_features="sqrt",
                                           subsample=0.8,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

40 0.8931
60 0.8931
80 0.8931
100 0.9203
120 0.8860
140 0.8918
160 0.8979
180 0.8994
200 0.8907
220 0.7739
240 0.8080
260 0.7752
280 0.8822
300 0.7757


In [41]:
for i in range(81, 120):
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.01, 
                                           n_estimators=64,
                                           min_samples_split=i,
                                           min_samples_leaf=40,
                                           max_depth=5,
                                           max_features="sqrt",
                                           subsample=0.8,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

81 0.9267
82 0.9242
83 0.8966
84 0.8817
85 0.8959
86 0.8992
87 0.9149
88 0.9242
89 0.8217
90 0.8159
91 0.9359
92 0.8964
93 0.8798
94 0.9274
95 0.8842
96 0.9143
97 0.9110
98 0.9304
99 0.9335
100 0.9203
101 0.9123
102 0.9130
103 0.8955
104 0.8943
105 0.8955
106 0.8927
107 0.8952
108 0.8837
109 0.8726
110 0.8952
111 0.9005
112 0.8863
113 0.8818
114 0.8607
115 0.8863
116 0.8839
117 0.8957
118 0.8912
119 0.9037


In [42]:
for i in range(10, 161, 10):
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.01, 
                                           n_estimators=64,
                                           min_samples_split=91,
                                           min_samples_leaf=i,
                                           max_depth=5,
                                           max_features="sqrt",
                                           subsample=0.8,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

10 0.9006
20 0.8989
30 0.8879
40 0.9359
50 0.9028
60 0.8942
70 0.8848
80 0.8764
90 0.8706
100 0.8319
110 0.7590
120 0.7985
130 0.6500
140 0.5000
150 0.1000
160 0.0000


In [43]:
for i in range(31, 50):
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.01, 
                                           n_estimators=64,
                                           min_samples_split=91,
                                           min_samples_leaf=i,
                                           max_depth=5,
                                           max_features="sqrt",
                                           subsample=0.8,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

31 0.8968
32 0.9073
33 0.9041
34 0.9040
35 0.9083
36 0.9240
37 0.9048
38 0.9071
39 0.8956
40 0.9359
41 0.8333
42 0.8767
43 0.9190
44 0.8774
45 0.9415
46 0.9465
47 0.9008
48 0.7959
49 0.8250


In [45]:
for i in range(5, 34, 2):
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.01, 
                                           n_estimators=64,
                                           min_samples_split=91,
                                           min_samples_leaf=46,
                                           max_depth=5,
                                           max_features=i,
                                           subsample=0.8,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

5 0.9465
7 0.8595
9 0.8767
11 0.8727
13 0.8672
15 0.8573
17 0.8577
19 0.8535
21 0.8572
23 0.8591
25 0.8524
27 0.8531
29 0.8481
31 0.8534
33 0.8531


In [46]:
for i in [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]:
    scores = []
    kfold = KFold(n_splits=10, random_state=54)
    for train_index, test_index in kfold.split(features.values):
        x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                            target.iloc[train_index], target.iloc[test_index]
        
        model = GradientBoostingClassifier(learning_rate=0.01, 
                                           n_estimators=64,
                                           min_samples_split=91,
                                           min_samples_leaf=46,
                                           max_depth=5,
                                           max_features=5,
                                           subsample=i,
                                           random_state=99)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores.append(precision_score(y_test, y_pred))
    
    print(i, '{0:.{1}f}'.format(sum(scores) / len(scores), 4))

0.6 0.8529
0.65 0.8864
0.7 0.8952
0.75 0.9220
0.8 0.9465
0.85 0.8749
0.9 0.8994
0.95 0.9454
1.0 0.8912


In [47]:
scores = []

model = GradientBoostingClassifier(learning_rate=0.01,
                                   n_estimators=64,
                                   max_depth=5,
                                   min_samples_split=91,
                                   min_samples_leaf=46,
                                   max_features=5,
                                   subsample=0.8,
                                   random_state=99)

cv = KFold(n_splits=10, random_state=32)
    
for train_index, test_index in cv.split(features.values):
    x_train, x_test, y_train, y_test = features.iloc[train_index], features.iloc[test_index], \
                                        target.iloc[train_index], target.iloc[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    scores.append(precision_score(y_test, y_pred))
    
print('Precision Score: {0:.{1}f}'.format(sum(scores) / len(scores), 2))

Precision Score: 0.95


<br>

***

<br>

<h1>7. Conclusion</h1><br>

The aim of this study is to predict customer churn and identify customers which will leave the company soon as accurately as possible.  
<br>
In Data Preparation stage, columns which are irrelevant was dropped and necessary feature transformation was made.  
<br>
In Exploratory Data Analysis stage, missing values in features was identified and replaced with appropriate values, outliers which may be occurred in features was detected. In addition, univariate and bivariate analysis was applied in order to observing distributions of features and relationships among attributes.  
<br>
In Data Preprocessing stage, binary features transformed into numerical, dummification of categorical features which have more than two distinct value was made and standard scaling was applied to numerical features in order to increase performance of machine learning models. Also, data was split as train and test for modeling.  
<br>
In Modeling stage, machine learning models related to classification was applied and their performance scores was printed.  
<br>
In Model Evaluation stage, performance metrics of models was compared each other and metrics which is most suitable for the problem was chosen. Furthermore, Cross-Validation was applied in order to exhibit the real performance of the models. It was understood that Gradient Boosting Tree has higher performance for this case. Afterwards, increasing the performance of the model was tried by tuning the parameters.  

<br>
Finally, the model gives precision score of 0.95 which means that thanks to this model 95% of the customers who will be chosen by the model and  will be presented new marketing campaign will be the actual customers who will quit doing business with the company.

<br>

***

<br>

<h1>8. References</h1><br>

* Lakshmanan, S., (2019, April 25). *Exploratory Data Analysis*. Retrieved from https://towardsdatascience.com/exploratory-data-analysis-in-python-ebdf643a33f6
<br><br>
* Swalin, A., (2018, May 3). *Choosing the Right Metric for Evaluating Machine Learning Models*. Retrieved from https://medium.com/usf-msds/choosing-the-right-metric-for-evaluating-machine-learning-models-part-2-86d5649a5428
<br><br>
* Boyle, T., (2019, Feb 4). *Dealing with Imbalanced Data*. Retrieved from https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18
<br><br>
* https://www.saedsayad.com/data_mining_map.htm

<br><br><br>